In [30]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb
from checklist.pred_wrapper import PredictorWrapper
import numpy as np
import random
import torch
from transformers import pipeline
import json
import requests
torch.manual_seed(0)
random.seed(0)
np.random.seed(0)
from scipy import spatial
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
from collections import Counter

torch.manual_seed(0)
random.seed(0)
np.random.seed(0)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
response = requests.get("https://square.ukp-lab.de/api/skill-manager/skill")
skills = response.json()
selected_skills = []
for skill in skills:
    if  "narrative" in skill['name'].lower():
        selected_skills.append(skill)
length_of_skills = len(selected_skills)
print(length_of_skills)

1


In [32]:
selected_skills

[{'id': '62c1ae1d536b1bb18ff91ce8',
  'name': 'NarrativeQA BART Adapter',
  'url': 'http://generative-qa',
  'skill_type': 'abstractive',
  'skill_settings': {'requires_context': True, 'requires_multiple_choices': 0},
  'user_id': 'ukp',
  'created_at': '2022-07-12T12:46:44.933479',
  'skill_input_examples': [{'query': '', 'context': '', 'answers': None},
   {'query': '', 'context': '', 'answers': None},
   {'query': '', 'context': '', 'answers': None}],
  'description': 'A multiple-choice QA skill with bart-base as base model with the narrativeqa adapter.',
  'default_skill_args': {'base_model': 'facebook/bart-base',
   'adapter': 'AdapterHub/narrativeqa'},
  'published': True,
  'client_id': 'ukp-NarrativeQA BART Adapter',
  'client_secret': None}]

In [33]:
data_file = "narrativeqa_selected.txt"
json_dicts = []
json_file_names = []
for i in range(length_of_skills):
    json_dicts.append(dict())
    json_dicts[i]["tests"] = []
    file_name = selected_skills[i]['id'] +'.json'
    json_file_names.append(file_name)

In [34]:
capabilities = {
    'Vocabulary': 'This capability tests whether a model has necessary vocabulary and whether it has the ability to handle the importance of different words.',
    'Taxonomy': 'This capability tests whether a model has the ability to handle synonyms, antonyms, object size, shape, profession etc.',
    'Robustness': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
    'NER': 'NER (Named Entity Recognition) Tests model’s ability to appropriately understand named entities.',
    'Negation': "This capability test's a model's ability to handle negation",
    'Temporal': "This capability test's a model's ability to understand order of events.",
    'Fairness': "This capability test's the model's gender(M/F) or racial bias",
    'Coref':"Coref (Coreference) Test's model's ability to appropriately understand he / she.",
    'SRL': "SRL (Semantic Role Labeling) Test's model's capability to understand different roles such as agents and objects."
    
}
test_types = {
    'MFT': 'MFT stands for Minimum Functionality Test. This testing type is inspired from unit testing of software engineering. For this type of testing precise and small testing datasets are created and the models are tested on that particular test set. MFTs are useful particularly for detecting when models use alternative approaches to handle complicated inputs without actually knowing the inside out of the capability. For MFT test cases, labeled test set is required.',
    'INV': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.'
}

In [35]:
def pred_answer(question, context, selected_skill_index):
    try:
        skill = selected_skills[selected_skill_index]
        skill_id = skill['id']
        query = {
            "query": question,
            "skill_args": {
                "context": context,
                "base_model": skill['default_skill_args']['base_model'],
                "adapter": skill['default_skill_args']['adapter'],
            },
        }
        
        headers = {'Content-type': 'application/json'}
        json_object = json.dumps(query)
        request_path = "https://square.ukp-lab.de/api/skill-manager/skill/"+skill_id+"/query"
        response = requests.post(request_path, data = json_object, headers = headers)
        predictions = response.json()
        return predictions['predictions'][0]['prediction_output']['output']
    except Exception as ex:
        print(ex)
        return "failed"

In [37]:
pred_answer("what's the saarne institute",'kate and john coleman vera farmiga and peter sarsgaard are experiencing strains in their marriage after their third child is stillborn the loss is particularly hard on kate who is also recovering from alcoholism the couple decides to adopt a 9yearold russian girl named esther isabelle fuhrman from a local orphanage while kate and johns deaf daughter max aryana engineer embraces esther immediately their son daniel jimmy bennett is less welcoming kate suspects that there might be problems in esthers background when esthers knowledge of sex extends beyond her age her suspicions deepen when esther injures another girl who had bullied her at school kate is further alarmed when sister abigail c c h pounder the head of the orphanage warns her that bad things happen when esther is around esther overhears this and plots to use max to distract sister abigail and then kill her esther pushes max into the path of sister abigails car forcing her to swerve off the road esther then kills sister abigail with a hammer and forces max to help her hide the weapon in their tree house kate is convinced that something is very wrong with esther but john does not believe her kate finds esthers hidden bible and discovers that it came from the saarne institute in estonia a mental hospital she emails a picture of esther to them and asks for more information when daniel learns about sister abigails death he tells max of his plan to retrieve the hammer to prove esthers guilt esther overhears this and confronts daniel at the tree house then sets it ablaze to kill him and destroy the evidence daniel falls from the tree trying to escape and is knocked unconscious esther attempts to finish him off but max stops her while daniel is hospitalized esther slips into his room and smothers him with a pillow stopping his heart doctors quickly revive him kate realizes what happened and attacks esther but is restrained and sedated that night a provocativelydressed esther tries to seduce a drunken john who finally realizes kate was right he tells esther that her future with the colemans is at stake upsetting her kate receives a call from dr v채rava karel roden the director of the saarne institute who reveals that esther is actually a 33yearold woman named leena klammer who has hypopituitarism a hormone disorder that stunted her physical growth and that she has spent most of her life posing as a little girl and had people adopt her with the goal of seducing the father and having a sexual relationship with him the doctor reports that leena has murdered at least seven people the previous family she was adopted by was killed by her because the father had rejected her sexual advances kate tries to get back home to stop esther and prevent her family from suffering the same fate leena flies into a rage after being spurned by john and ransacks her room removing her little girl makeup revealing the scars from a straitjacket and weathered skin leena stabs and kills john max witnesses this and hides kate rushes home and finds john dead leena gets a gun from johns safe and shoots kate in the arm kate manages to take the gun and flee with max leena finds kate and max near a frozen pond she lunges at kate hurling them both into the ice kate climbs partially out of the pond leena reverting to her esther persona begs kate not to let her die while hiding a knife behind her back kate kills leena by kicking her in the face breaking her neck max and kate are met by the police moments after', 0)

[Errno Expecting value] Internal Server Error: 0


'failed'

In [20]:
model = SentenceTransformer('all-MiniLM-L6-v2')
thresholds = [0.5, 0.5, 0.7,0.5]

def jaccard_similarity(x, y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

def tfidf_similarity(corpus):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    return cosine_sim[0][1]

def sbert_similarity(sentence, changed_sentence):
    sentences = [sentence, changed_sentence]
    embeddings = model.encode(sentences)
    result = 1 - spatial.distance.cosine(embeddings[0], embeddings[1])
    return result

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    #def remove_articles(text):
    #    return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(s)))


def f1_score(ground_truth, prediction):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def answer_or_not(sentence1, sentence2, thresholds):
    sentence1 = normalize_answer(sentence1)
    sentence2 = normalize_answer(sentence2)
    corpus = [sentence1, sentence2]
    s1_splitted = sentence1.split(" ")
    s2_splitted = sentence2.split(" ")
    scores = []
    scores.append(jaccard_similarity(s1_splitted, s2_splitted))
    scores.append(tfidf_similarity(corpus))
    scores.append(sbert_similarity(sentence1, sentence2))
    scores.append(f1_score(sentence1, sentence2))
    got_it = []
    for i in range(len(scores)):
        if scores[i]>thresholds[i]:
            got_it.append(1)
        else:
            got_it.append(0)
    if sum(got_it) >= 3:
        return 1
    else:
        return 0

In [21]:
nq_open = open(data_file, 'r')
questions = []
answers_nq = []
contexts_na = []
for line in nq_open:
    line = line.strip()
    elements = line.split("|||||")
    questions.append(normalize_answer(elements[0]))
    answers_nq.append(normalize_answer(elements[2]))
    contexts_na.append(normalize_answer(elements[1]))

In [22]:
questions[10]

'what are people arguing about as the story begins'

In [23]:
def get_answer(question):
    for i in range(len(questions)):
        if questions[i].strip() == question.strip():
            return answers_nq[i], contexts_na[i]

In [24]:
contraction_map = {
            "isn't": "is not", "ain't": "is not", "aren't": "are not", "can't": "cannot",
            "can't've": "cannot have", "could've": "could have", "couldn't":
            "could not", "didn't": "did not", "doesn't": "does not", "don't":
            "do not", "hadn't": "had not", "hasn't": "has not", "haven't":
            "have not", "he'd": "he would", "he'd've": "he would have",
            "he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y":
            "how do you", "how'll": "how will", "how's": "how is",
            "I'd": "I would", "I'll": "I will", "I'm": "I am",
            "I've": "I have", "i'd": "i would", "i'll": "i will",
            "i'm": "i am", "i've": "i have",
            "it'd": "it would", "it'll": "it will", "it's": "it is", "ma'am":
            "madam", "might've": "might have", "mightn't": "might not",
            "must've": "must have", "mustn't": "must not", "needn't":
            "need not", "oughtn't": "ought not", "shan't": "shall not",
            "she'd": "she would", "she'll": "she will", "she's": "she is",
            "should've": "should have", "shouldn't": "should not", "that'd":
            "that would", "that's": "that is", "there'd": "there would",
            "there's": "there is", "they'd": "they would",
            "they'll": "they will", "they're": "they are",
            "they've": "they have", "wasn't": "was not", "we'd": "we would",
            "we'll": "we will", "we're": "we are", "we've": "we have",
            "weren't": "were not", "what're": "what are", "what's": "what is",
            "when's": "when is", "where'd": "where did", "where's": "where is",
            "where've": "where have", "who'll": "who will", "who's": "who is",
            "who've": "who have", "why's": "why is", "won't": "will not",
            "would've": "would have", "wouldn't": "would not",
            "you'd": "you would", "you'd've": "you would have",
            "you'll": "you will", "you're": "you are", "you've": "you have"
}



In [25]:
def contractions(x):
    conts = Perturb.contractions(x)
    return conts
t = Perturb.perturb(questions, contractions, nsamples=5)

In [26]:
t.data

[['what is john buckinghams job', "what's john buckinghams job"],
 ['what is the cavendish estate in england essex county known as',
  "what's the cavendish estate in england essex county known as"],
 ['what is the name of jacks lover in the beginning of the story',
  "what's the name of jacks lover in the beginning of the story"],
 ['why is grassou still resentful despite his potentially advantageous marriage',
  "why's grassou still resentful despite his potentially advantageous marriage"],
 ['who is the main artist of the story', "who's the main artist of the story"]]

In [27]:
import re
keys = list(contraction_map.keys())
values = list(contraction_map.values())
changes = []
for i in range(len(t.data)):
    actual_sentence =  t.data[i][0]
    changed_sentence = t.data[i][1]
    #print(actual_sentence)
    #print(changed_sentence)
    key_words = [] 
    for k in keys:
        r = re.search(k,changed_sentence, re.IGNORECASE)
        if bool(r) == True and r.group() not in key_words:
            if changed_sentence[r.span(0)[1]] == ' ':
                key_words.append(r.group())

    value_words = []
    for k in values:
        r = re.search(k, changed_sentence, re.IGNORECASE)
        if bool(r) == True and r.group() not in value_words:
            if changed_sentence[r.span(0)[1]] == ' ':
                value_words.append(r.group())
    all_words = key_words + value_words
    #print(all_words)
    all_values = []
    all_values_ = []
    for w in all_words:
        if w in keys:
            all_values.append(contraction_map[w])
        elif w.capitalize() in keys:
            all_values.append(contraction_map[w.capitalize()])
        elif w.lower() in keys:
            all_values.append(contraction_map[w.lower()])
        elif w in values:
            for j in range(len(values)):
                if w.strip() == values[j] and keys[j] not in all_values:
                    all_values.append(keys[j])
        elif w.capitalize() in values:
            for j in range(len(values)):
                if w.capitalize() == values[j] and keys[j] not in all_values:
                    all_values.append(keys[j])
        elif w.lower() in values:
            for j in range(len(values)):
                if w.lower() == values[j] and keys[j] not in all_values:
                    all_values.append(keys[j])
    actual = []
    change = []
    for i in range(len(all_words)):
        try:
            r = re.search(all_values[i],actual_sentence, re.IGNORECASE)
            if r.group() not in all_values_:
                actual.append(r.group())
            change.append(all_words[i])
        except:
            continue
    changed_pairs = []
    for m in range(len(actual)):
        o, c = actual[m], change[m]
        changed_pairs.append((o, c))
    changes.append(changed_pairs)

In [28]:
def add_test(data, changes, test_name, capability, test_type, name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q = data[i][0]
            changed_q = data[i][1]
            change = changes[i]
            answer, context = get_answer(original_q)
            a = pred_answer(original_q, context, skill_index)
            a = normalize_answer(a.strip().lower())
            p = pred_answer(changed_q, context, skill_index)
            p = normalize_answer(p.strip().lower())
            test_case_dict = dict()
            total_cases = total_cases + 1 
            test_case_dict["original_question"] = original_q
            test_case_dict["question"] = changed_q
            test_case_dict["context"] = context
            answer, context = get_answer(original_q)
            test_case_dict["gold_answer"] = answer
            test_case_dict["prediction_before_change"] = a
            test_case_dict["prediction"] = p
            test_case_dict["prediction"] = p
            changed_dict = dict()
            changed_dict["from"] = []
            changed_dict["to"] = []
            for n in range(len(change)):
                source_target_pairs = change[n]
                s, t = source_target_pairs
                changed_dict["from"].append(s)
                changed_dict["to"].append(t)
            test_case_dict["changed"] = changed_dict
            feedback = answer_or_not(p, a, thresholds)
            if feedback == 0:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                test_case_dict["success_failed"] = "success"
            '''
            if a != p:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                if a == "Failed" and p == "Failed":
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
            '''
            test_case_dict["where"] = "question"
            test_cases.append(test_case_dict)
            
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)

    return 0

In [29]:
name='Question contractions'
test_type = "INV"
capability='Robustness'
thresholds = [0.25, 0.25, 0.5, 0.25]
n_description = "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(t.data, changes, name, capability, test_type, n_description, t_description, c_description)

bank clerk
[Errno Expecting value] Internal Server Error: 0


KeyboardInterrupt: 

In [18]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'what is the saarne institute',
    'question': "what's the saarne institute",
    'context': 'kate and john coleman vera farmiga and peter sarsga

In [186]:
def addtypos(x):
    conts = Perturb.add_typos(x)
    return conts
t = Perturb.perturb(questions, addtypos, nsamples=10)

In [187]:
t.data

[['what did paul suffer from', 'what did paul suffe rfrom'],
 ['who is the 2nd earl of essex', 'who is the 2nd ealr of essex'],
 ['who is jacks lover', 'who is jack slover'],
 ['how much money do pearce and elise get away with',
  'ohw much money do pearce and elise get away with'],
 ['what disease does kate suffer from', 'wha tdisease does kate suffer from'],
 ['what does john removing leenas makeup reveal',
  'what does john removing leenas makeu preveal'],
 ['which page performs the dialogue', 'which page performs hte dialogue'],
 ['what was john tied to in the motel', 'what was john tied ot in the motel'],
 ['which disability is max afflicted with',
  'whichd isability is max afflicted with'],
 ['what is john buckinghams job', 'what is john buckingahms job']]

In [188]:
changed_pairs = []
for i in range(len(t.data)):
   
    actual_sentence = t.data[i][0]
    changed_sentence = t.data[i][1]
    actual_list = actual_sentence.split()
    changed_list = changed_sentence.split()
    a_list = []
    for j in range(len(changed_list)):
        a = actual_list[j]
        c = changed_list[j]
        if a != c:
            pair = (a, c)
            a_list.append(pair)
    changed_pairs.append(a_list)

In [189]:
name='Question typo'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle questions typos (whether changing the spelling of words in the questions changes the model's output)"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(t.data, changed_pairs, name, capability, test_type, n_description, t_description, c_description)

a severe bladder infection
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
robert devereux
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
ingrid
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
23 billion
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
alcoholism
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
scars from a strait jacket and weathered skin
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
anaides
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
toilet
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value] Internal Server Error: 0
she is deaf
[Errno Expecting value] Internal Server Error: 0
[Errno Expecting value

0

In [22]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'what is the saarne institute',
    'question': "what's the saarne institute",
    'context': 'kate and john coleman vera farmiga and peter sarsga

In [23]:
nlp = spacy.load('en_core_web_sm')

In [24]:
from datasets import load_dataset
dataset = load_dataset('squad_v2', split='validation')
new_questions = [d['question'] for d in dataset]
contexts = [d['context'] for d in dataset]
answers = []
i = 0
pairs = []
for single_data in dataset:
    try:
        c = single_data["context"]
        q = single_data["question"]
        a = single_data["answers"]["text"]
        if bool(a[0].strip()) == True:
            pairs.append((c, q))
            answers.append(a)
    except:
        i = i+1
        #answers.append(["Not a Answerable Question"])
print(i)

Reusing dataset squad_v2 (/home/saadi/.cache/huggingface/datasets/squad_v2/squad_v2/2.0.0/09187c73c1b837c95d9a249cd97c2c3f1cebada06efe667b4427714b27639b1d)


5945


In [25]:
from tqdm import tqdm
contexts = []
new_questions = []
for p in pairs:
    c, q = p
    contexts.append(c)
    new_questions.append(q)
qa = list(zip(new_questions, contexts))

all_qa = set()
all_qa.update(set(new_questions))
all_qa.update(set(contexts))

processed_qa = list(tqdm(nlp.pipe(all_qa, batch_size=64), total=len(all_qa)))
spacy_map = {q: processed_q for (q, processed_q) in zip(all_qa, processed_qa)}
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in pairs]

100%|██████████████████████████████████████| 7130/7130 [00:22<00:00, 321.74it/s]


In [26]:
random_sentences = set()
for x, _ in processed_pairs:
    for y in x.sents:
        random_sentences.add(y.text)
random_sentences = list(random_sentences)

In [27]:
def add_random_sentence(x, **kwargs):
    random_s = np.random.choice(random_sentences)
    while random_s in x[0]:
        random_s = np.random.choice(random_sentences)
    random_s = random_s.strip('.') + '. '
    meta = ['add to end: %s' % random_s, 'add to beg: %s' % random_s]
    return [x +  " " + random_s, random_s + " " + x], meta

In [28]:
t = Perturb.perturb(questions , add_random_sentence, nsamples=2, meta=True)

In [29]:
def add_random_sentence_test(data, changes, test_name, capability, test_type, 
                             name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["c_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q = data[i][0]
            temp_data = data[i][1:]
            temp_changes = changes[i][1:]
            for j in range(len(temp_data)):
                changed_q = temp_data[j]
                random_sentence = temp_changes[j] 
                answer, context = get_answer(original_q)
                a = pred_answer(original_q, context, skill_index)
                a = normalize_answer(a.strip().lower())
                p = pred_answer(changed_q, context, skill_index)
                p = normalize_answer(p.strip().lower())
                test_case_dict = dict()
                total_cases = total_cases + 1  
                test_case_dict["original_question"] = original_q
                test_case_dict["question"] = changed_q
                test_case_dict["context"] = context
                test_case_dict["prediction_before_change"] = a
                test_case_dict["gold_answer"] = answer
                test_case_dict["prediction"] = p
                temp = changed_q.replace(original_q, '')
                start = changed_q.find(temp)
                end = start + len(temp)
                span = [start, end]
                test_case_dict["span"] = span
                feedback = answer_or_not(p, a, thresholds)
                if feedback == 0:
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
                test_case_dict["where"] = "question"
                test_cases.append(test_case_dict)
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)
    return 0

In [30]:
name = 'Add random sentence to context'
capability='Robustness'
test_type="INV"
n_description = "Test's model's ability to handle addition of random sentences in the model's input context (whether addition of a random sentence at the beginning and end of the input context changes model's output)."
c_description = capabilities[capability]
t_description = test_types[test_type]
add_random_sentence_test(t.data, t.meta, name, capability, test_type, n_description, t_description, c_description)

0

In [31]:
json_dicts[0]

{'tests': [{'test_type': 'INV',
   'capability': 'Robustness',
   'test_name': 'Question contractions',
   'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
   'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
   'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
   'test_cases': [{'original_question': 'what is the saarne institute',
     'question': "what's the saarne institute",
     'context': 'kate and john coleman vera farmig

In [32]:
new_questions = questions.copy()
for i in range(len(new_questions)):
    new_questions[i] = new_questions[i].upper()
fake_context = contexts_na.copy()

In [33]:
fake_pairs = []
for fc, q in zip(fake_context, new_questions):
    fake_pairs.append((fc, q))

In [34]:
from tqdm import tqdm
contexts = []
questions_ = []
for p in fake_pairs:
    c, q = p
    contexts.append(c.upper())
    questions_.append(q.upper())
#pairs = [(x['passage'], x['question']) for x in data]
qa = list(zip(questions_, contexts))

all_qa = set()
all_qa.update(set(questions_))
all_qa.update(set(contexts))

processed_qa = list(tqdm(nlp.pipe(all_qa, batch_size=64), total=len(all_qa)))
spacy_map = {q: processed_q for (q, processed_q) in zip(all_qa, processed_qa)}
#processed_pairs = [(spacy_map[q], spacy_map[c]) for q, c in qa]
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in qa]

100%|████████████████████████████████████████| 171/171 [00:01<00:00, 159.59it/s]


In [35]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both
            

In [36]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_names), nsamples=2, meta=True)

In [37]:
t.data

[[('WHO DOES MARY MARRY',
   'MARY BEGINS WITH A DESCRIPTION OF THE CONVENTIONAL AND LOVELESS MARRIAGE BETWEEN THE HEROINES MOTHER AND FATHER ELIZA MARYS MOTHER IS OBSESSED WITH NOVELS RARELY CONSIDERS ANYONE BUT HERSELF AND FAVOURS MARYS BROTHER SHE NEGLECTS HER DAUGHTER WHO EDUCATES HERSELF USING ONLY BOOKS AND THE NATURAL WORLD IGNORED BY HER FAMILY MARY DEVOTES MUCH OF HER TIME TO CHARITY WHEN HER BROTHER SUDDENLY DIES LEAVING MARY HEIR TO THE FAMILYS FORTUNE HER MOTHER FINALLY TAKES AN INTEREST IN HER SHE IS TAUGHT ACCOMPLISHMENTS SUCH AS DANCING THAT WILL ATTRACT SUITORS HOWEVER MARYS MOTHER SOON SICKENS AND REQUESTS ON HER DEATHBED THAT MARY WED CHARLES A WEALTHY MAN SHE HAS NEVER MET STUNNED AND UNABLE TO REFUSE MARY AGREES IMMEDIATELY AFTER THE CEREMONY CHARLES DEPARTS FOR THE CONTINENT TO ESCAPE A FAMILY WHO DOES NOT SHARE HER VALUES MARY BEFRIENDS ANN A LOCAL GIRL WHO EDUCATES HER STILL FURTHER MARY BECOMES QUITE ATTACHED TO ANN WHO IS IN THE GRIP OF AN UNREQUITED LOVE AND D

In [38]:
t.meta

[[None,
  ('MARY', 'Courtney'),
  ('MARY', 'Vanessa'),
  ('MARY', 'Jacqueline'),
  ('MARY', 'Alicia'),
  ('MARY', 'Angela'),
  ('MARY', 'Emily'),
  ('MARY', 'Melissa'),
  ('MARY', 'Sarah'),
  ('MARY', 'Sara'),
  ('MARY', 'Jacqueline')],
 [None,
  ('JACK', 'Kyle'),
  ('JACK', 'Sean'),
  ('JACK', 'Ian'),
  ('JACK', 'Paul'),
  ('JACK', 'Patrick'),
  ('JACK', 'Edward'),
  ('JACK', 'Dustin'),
  ('JACK', 'Patrick'),
  ('JACK', 'Zachary'),
  ('JACK', 'David')]]

In [39]:
def add_ner(data, changes, test_name, capability, test_type, 
            name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q, context = data[i][0]
            original_q, o_context = original_q.lower(), context.lower()
            answer, _ = get_answer(original_q)
            temp_data = data[i][1:]
            temp_changes = changes[i][1:]
            a = pred_answer(original_q, o_context, skill_index)
            a = normalize_answer(a.strip().lower())
            for j in range(len(temp_data)):
                changed_q, context = temp_data[j]
                changed_q, c_context = changed_q.lower(), context.lower()
                s, t = temp_changes[j]
                s, t = s.lower(), t.lower()
                p = pred_answer(changed_q, c_context, skill_index)
                p = normalize_answer(p.strip().lower())
                test_case_dict = dict()
                total_cases = total_cases + 1  
                test_case_dict["original_question"] = original_q
                test_case_dict["original_context"] = o_context
                test_case_dict["question"] = changed_q
                test_case_dict["context"] = c_context
                test_case_dict["prediction_before_change"] = a
                test_case_dict["gold_answer"] = answer
                test_case_dict["prediction"] = p
                changed_dict = dict()
                changed_dict["from"] = s
                changed_dict["to"] = t
                test_case_dict["changed"] = changed_dict
                feedback = answer_or_not(p, a, thresholds)
                if feedback == 0:
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
                '''
                if a != p:
                    temp_p = p
                    if s == a and t == p:
                        test_case_dict["success_failed"] = "success"
                    elif s in a and temp_p.replace(t,s) == a:
                        test_case_dict["success_failed"] = "success"
                    else:
                        failed_cases = failed_cases + 1
                        test_case_dict["success_failed"] = "failed"
                else:
                    if a == "Failed" and p == "Failed":
                        failed_cases = failed_cases + 1
                        test_case_dict["success_failed"] = "failed"
                    else:
                        test_case_dict["success_failed"] = "success"
                '''
                #test_case_dict["where"] = "question"
                test_cases.append(test_case_dict)
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)
    return 0

In [40]:
name = 'Change name everywhere'
capability='NER'
test_type="INV"
n_description = "Test's model's capability to handle changing a name in the input context and also in the question (if it contains the name)(whether a name change in the context and the questions changes the model output)."
c_description = capabilities[capability]
t_description = test_types[test_type]
add_ner(t.data, t.meta, name, capability, test_type, n_description, t_description, c_description)

0

In [41]:
json_dicts[0]

{'tests': [{'test_type': 'INV',
   'capability': 'Robustness',
   'test_name': 'Question contractions',
   'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
   'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
   'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
   'test_cases': [{'original_question': 'what is the saarne institute',
     'question': "what's the saarne institute",
     'context': 'kate and john coleman vera farmig

In [42]:
from tqdm import tqdm
contexts = []
questions_ = []
for p in fake_pairs:
    c, q = p
    contexts.append(c.title())
    questions_.append(q.title())
#pairs = [(x['passage'], x['question']) for x in data]
qa = list(zip(questions_, contexts))

all_qa = set()
all_qa.update(set(questions_))
all_qa.update(set(contexts))

processed_qa = list(tqdm(nlp.pipe(all_qa, batch_size=64), total=len(all_qa)))
spacy_map = {q: processed_q for (q, processed_q) in zip(all_qa, processed_qa)}
#processed_pairs = [(spacy_map[q], spacy_map[c]) for q, c in qa]
processed_pairs = [(spacy_map[x[0]], spacy_map[x[1]]) for x in qa]

100%|████████████████████████████████████████| 171/171 [00:01<00:00, 161.25it/s]


In [43]:
import re
def change_thing(change_fn):
    def change_both(cq, **kwargs):
        context, question = cq
        a = change_fn(context, meta=True)
        if not a:
            return None
        changed, meta = a
        ret = []
        for c, m in zip(changed, meta):
            new_q = re.sub(r'\b%s\b' % re.escape(m[0]), m[1], question.text)
            ret.append((c, new_q))
        return ret, meta
    return change_both
            

In [44]:
t = Perturb.perturb(processed_pairs, change_thing(Perturb.change_location), nsamples=1, meta=True)

In [45]:
t.data

[[('How Much Heavier Than Micky Is Mickys Opponent In Atlantic City',
   'On The Night Of An Undercard Fight In Atlantic City Mickys Scheduled Opponent Is Ill And A Substitute Is Found Who Is 20 Pounds Heavier Than Micky A Huge Difference In Professional Boxing Constituting Two Or Three Weight Classes Despite Mickys Reservations His Mother And Brother Agree So That They Can All Get The Purse And Micky Is Defeated Micky Retreats From The World And Forms A Relationship With Charlene Fleming A Former College Athlete Who Dropped Out And Became A Bartender After Several Weeks Alice Arranges Another Fight For Micky Concerned It Will Turn Out The Same His Mother And Seven Sisters Blame Charlene For His Lack Of Motivation Micky Mentions He Received An Offer To Be Paid To Train In Las Vegas But Dicky Says He Will Match The Offer So He Can Keep Training And Working With His Family Dicky Then Tries To Get Money By Posing His Girlfriend As A Prostitute And Then Once She Picks Up A Client Impersona

In [46]:
t.meta

[[None,
  ('Atlantic City', 'Louisville/Jefferson County'),
  ('Atlantic City', 'Cleveland'),
  ('Atlantic City', 'Indianapolis'),
  ('Atlantic City', 'Garland'),
  ('Atlantic City', 'Chandler'),
  ('Atlantic City', 'Milwaukee'),
  ('Atlantic City', 'Chesapeake'),
  ('Atlantic City', 'Sacramento'),
  ('Atlantic City', 'Aurora'),
  ('Atlantic City', 'Durham')]]

In [47]:
name = 'Change location everywhere'
capability='NER'
test_type="INV"
n_description = "Test's model's capability to handle changing a location in the input context and also in the question (if it contains the name)(whether a location change in the context and the question changes the model output)."
c_description = capabilities[capability]
t_description = test_types[test_type]
add_ner(t.data, t.meta, name, capability, test_type, n_description, t_description, c_description)

0

In [48]:
json_dicts[0]

{'tests': [{'test_type': 'INV',
   'capability': 'Robustness',
   'test_name': 'Question contractions',
   'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
   'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
   'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
   'test_cases': [{'original_question': 'what is the saarne institute',
     'question': "what's the saarne institute",
     'context': 'kate and john coleman vera farmig

In [49]:
from transformers import BartForConditionalGeneration, BartTokenizer
paraphraser = BartForConditionalGeneration.from_pretrained('eugenesiow/bart-paraphrase')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
paraphraser = paraphraser.to(device)
tokenizer = BartTokenizer.from_pretrained('eugenesiow/bart-paraphrase')

In [50]:
i = 0
paraphrased_questions = []
for q in questions:
    batch = tokenizer(q, return_tensors='pt')
    generated_ids = paraphraser.generate(batch['input_ids'])
    generated_sentence = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    generated_sentence = generated_sentence.replace('?', '').lower()
    if q.strip() == generated_sentence:
        continue
    result = sbert_similarity(q, generated_sentence)
    if result >= 0.95:
        paraphrased_questions.append([q, generated_sentence])
        i = i + 1
    else:
        continue
    if i == 50:
        break

In [51]:
paraphrased_questions

[['what did the symbolic vices disguise themselves to be',
  'what did symbolic vices disguise themselves to be'],
 ['how many phases does the competition have',
  'how many phases does competition have'],
 ['which page performs the dialogue', 'which page performs dialogue'],
 ['what does diana order to take place in the valley of gargaphie',
  'what does diana do to take place in the valley of gargaphie'],
 ['who awakens echo', 'who awakens the echo'],
 ['who does cynthia represent in the masque', 'who is cynthia in the masque'],
 ['what reward does micky gain in the final victory',
  'what rewards does micky gain in the final victory'],
 ['who suffers from a crack addiction',
  'who is suffering from a crack addiction'],
 ['why did john want to take nadia to the police after yuri and alexei left with the money',
  'why did john want to take nadia to the police after yuri and alexei'],
 ['what country did nadias cousin and friend visit her from',
  'which country did nadias cousin and

In [52]:
paraphrased_questions= [#['what did the symbolic vices disguise themselves to be',
  #'what did symbolic vices disguise themselves to be'],
 #['how many phases does the competition have',
 # 'how many phases does competition have'],
 #['which page performs the dialogue', 'which page performs dialogue'],
 #['what does diana order to take place in the valley of gargaphie',
 # 'what does diana do to take place in the valley of gargaphie'],
 #['who awakens echo', 'who awakens the echo'],
 #['who does cynthia represent in the masque', 'who is cynthia in the masque'],
 #['what reward does micky gain in the final victory',
 # 'what rewards does micky gain in the final victory'],
 ['who suffers from a crack addiction',
  'who is suffering from a crack addiction'],
 #['why did john want to take nadia to the police after yuri and alexei left with the money',
 # 'why did john want to take nadia to the police after yuri and alexei'],
 ['what country did nadias cousin and friend visit her from',
  'which country did nadias cousin and friend visit her from'],
 ['what was john tied to in the motel', 'what was john tied up in the motel'],
 #['what is john buckinghams job', "what is john buckingham's job"],
 ['what crime was john coffey convicted for',
  'what crime was john coffey convicted of'],
 #['why did john coffey watch top hat',
 # 'why did john coffey watch the top hat'],
 #['who cries while watching top hat in 1999',
 # 'who cried while watching top hat in 1999'],
 #['who owes millions in taxes', 'who owes millions of dollars in taxes'],
 #['where is elises suite', 'where is the elises suite'],
 #['how much money do pearce and elise get away with',
 # 'how much money did pearce and elise get away with'],
 #['what is seen leaving jacks car in the final scene',
 # 'what is seen leaving the jacks car in the final scene'],
 #['why does jack sabotage the rifle', 'why does the jack sabotage the rifle'],
 #['at the end of the story what flees from jacks car',
 # 'at the end of the story, what escapes from the jacks car'],
 #['what kind of paintings does pierre paint',
 # 'what kind of painting does pierre paint do'],
 ['what is the fatal thought that grassou is believed unable to remove from his heart',
  'what is the fatal thought that grassou is unable to remove from his heart'],
 #['why is grassou still unhappy even after marriage',
 # 'why is grassou still unhappy after marriage'],
 ['where do kate and max end up after they flee the house',
  'where do kate and max end up after they escape the house'],
 #['what is the saarne institute', 'what is saarne institute'],
 #['what does dr varava reveal about esther',
 # 'what does dr. varava reveal about esther'],
 ['who gets captured by the picts and tied up',
  'who is captured by the picts and tied up'],
 #['who warns settlers of the pictish attacks',
 # 'who warns settlers of the pictish attacks.'],
 #['why is conan marked for death by the gods of darkness',
 # 'why was conan marked for death by the gods of darkness'],
 ['what is balthus slaying when conan meets him in the forest',
  'what is balthus slaying when conan meets him in the woods'],
 #['on what river was fort tuscelan',
 # 'on what river was fort tuscelan located'],
 #['how many pictish heads does conan vow to take for slasher',
 # 'how many pictish heads does conan vow to take for the slasher'],
 #['who warns the women and children to flee',
 # 'who warns the women and children to flee.'],
 #['when mary was younger how did her mother fill her free time',
 # 'when mary was younger, how did her mother fill her free time'],
 #['prior to her brothers death how did mary spend most of her time outside of the home',
 # 'prior to her brothers death, how did mary spend most of her time outside of the home'],
 #['what does mary do when she hears that henrys consumption has worsened',
 # 'what does mary do when she finds out that henrys consumption has worsened'],
 ['where is the school located where roberta takes a position as a substitute violin teacher',
  'where is the school where roberta takes a position as a substitute violin teacher'],
 #['why does roberta lose her teaching job',
 # 'why did roberta lose her teaching job'],
 ['in the midst of her deep despair who was it that encouraged roberta to move on with her life',
  'in the midst of her deep despair, who encouraged roberta to move on with her'],
 #['not all efforts were smoth sailing in planning the benefit what specifically caused an upset',
 # 'not all efforts were smoth sailing in planning the benefit what specifically caused an upset.'],
 #['though the benefit concert was well received what was the ultimate outcome',
 # 'though the benefit concert was well received, what was the ultimate outcome']
]

In [53]:
len(paraphrased_questions)

10

In [54]:
def add_paraphrased_test(data, test_name, capability, test_type, name_description, type_description, c_description):
    for skill_index in range(len(selected_skills)):
        failed_cases = 0
        total_cases = 0
        test_dict = dict()
        test_dict["test_type"] = test_type
        test_dict["capability"] = capability
        test_dict["test_name"] = test_name
        test_dict["test_name_description"] = name_description
        test_dict["test_type_description"] = type_description
        test_dict["capability_description"] = c_description
        test_cases = []
        for i in range(len(data)):
            original_q, changed_q= data[i][0], data[i][1]
            answer, context = get_answer(original_q)
            a = pred_answer(original_q, context, skill_index)
            a = normalize_answer(a.strip().lower())
            p = pred_answer(changed_q, context, skill_index)
            p = normalize_answer(p.strip().lower())
            test_case_dict = dict()
            total_cases = total_cases + 1 
            test_case_dict["original_question"] = original_q
            test_case_dict["question"] = changed_q
            test_case_dict["context"] = context
            test_case_dict["gold_answer"] = answer
            test_case_dict["prediction_before_change"] = a
            test_case_dict["prediction"] = p
            feedback = answer_or_not(p, a, thresholds)
            if feedback == 0:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                test_case_dict["success_failed"] = "success"
            '''
            if a != p:
                failed_cases = failed_cases + 1
                test_case_dict["success_failed"] = "failed"
            else:
                if a == "Failed" and p == "Failed":
                    failed_cases = failed_cases + 1
                    test_case_dict["success_failed"] = "failed"
                else:
                    test_case_dict["success_failed"] = "success"
            '''
            test_case_dict["where"] = "question"
            test_cases.append(test_case_dict)
        failure_rate = (failed_cases / total_cases)*100
        failure_rate_ = "{:.2f}".format(failure_rate )
        failure_rate = float(failure_rate_)
        success_rate = 100 - failure_rate
        test_dict["test_cases"] = test_cases
        test_dict["total_cases"] = total_cases
        test_dict["failed_cases"] = failed_cases
        test_dict["success_cases"] = total_cases - failed_cases
        test_dict["failure_rate"] = failure_rate
        test_dict["success_rate"] = success_rate
        json_dicts[skill_index]["tests"].append(test_dict)
    return 0

In [55]:
name = 'Paraphrase Question'
capability='Robustness'
test_type="INV"
n_description = "Test's model's capability to handle paraphrased question"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_paraphrased_test(paraphrased_questions, name, capability, test_type, n_description, t_description, c_description)

0

In [56]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'what is the saarne institute',
    'question': "what's the saarne institute",
    'context': 'kate and john coleman vera farmiga and peter sarsga

In [57]:
def shuffle_question(question):
    random.seed(0)
    question = question.replace('?','')
    question = question.replace('.','')
    splitted_question = question.split()
    word_range = range(0,len(splitted_question))
    selected_indexes = random.sample(word_range, 2)
    word = splitted_question[selected_indexes[0]]
    word = wordword = splitted_question[selected_indexes[0]]
    splitted_question[selected_indexes[0]] = splitted_question[selected_indexes[1]]
    splitted_question[selected_indexes[1]] = word
    return ' '.join(splitted_question)

In [58]:
nsamples = 10
i = 0
changes = []
shuffuled_questions = []
j = 0
for i in range(len(questions)):
    q = questions[i]
    c_q = shuffle_question(q)
    result = sbert_similarity(q, c_q)
    print(q)
    print(c_q)
    c_in_q = []
    q_split = q.split()
    c_split = c_q.split()
    for n in range(len(q.split())):
        r = q_split[n]
        c = c_split[n]
        if r != c:
            c_in_q.append((r, c))
    if result >= 0.92:
        shuffuled_questions.append([q, c_q])
        changes.append(c_in_q)
        j = j + 1
    if j == nsamples:
        break

what name was cynthia more famously known by
what name was cynthia more famously by known
in what valley did the solemn revels of cynthia take place
in what valley did the solemn place of cynthia take revels
what did the symbolic vices disguise themselves to be
what did the symbolic vices disguise be to themselves
what sentence did cynthia give to the symbolic vices
what sentence did cynthia give to vices symbolic the
who drank from the spring at cynthias revels
who drank from the spring at revels cynthias
how many phases does the competition have
how many phases have the competition does
who does diana represent
who represent diana does
where do the dancers purify themselves
where do the themselves purify dancers
which page performs the dialogue
which page performs dialogue the
who is the 2nd earl of essex
who is the essex earl of 2nd


In [59]:
changes

[[('known', 'by'), ('by', 'known')],
 [('revels', 'place'), ('place', 'revels')],
 [('themselves', 'be'), ('be', 'themselves')],
 [('the', 'vices'), ('vices', 'the')],
 [('cynthias', 'revels'), ('revels', 'cynthias')],
 [('does', 'have'), ('have', 'does')],
 [('does', 'represent'), ('represent', 'does')],
 [('dancers', 'themselves'), ('themselves', 'dancers')],
 [('the', 'dialogue'), ('dialogue', 'the')],
 [('2nd', 'essex'), ('essex', '2nd')]]

In [60]:
name='Perturb some words'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle unorderd questions "
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(shuffuled_questions, changes, name, capability, test_type, n_description, t_description, c_description)

the goddess diana
gargaphie
virtues
make reperations and purify themselves
the courtiers and ladies that are there and asotus
four
queen elizabeth
in the spring at mount helicon
anaides
robert devereux


0

In [61]:
def process_questions(ques):
    changes = []
    ans = []
    questions_ = []
    for i in range(len(ques)):
        q = ques[i]
        elements = q.split("|||||")
        splited = elements[0].split()
        oq = []
        cq = []
        change = []
        for j in range(len(splited)):
            if "/" not in splited[j]:
                oq.append(splited[j])
                cq.append(splited[j])
            else:
                temp = splited[j].split("/")
                oq.append(temp[0])
                cq.append(temp[1])
                change.append((temp[0], temp[1]))
        original = ' '.join([str(w) for w in oq])
        changed = ' '.join([str(w) for w in cq])
        questions_.append([original, changed])
        changes.append(change)
        ans.append(elements[1:])
    return questions_, changes, ans

In [62]:
#synonym questions nq
synonyms = ["what title does micky finally/ultimately win|||||on the night of an undercard fight in atlantic city mickys scheduled opponent is ill and a substitute is found who is 20 pounds heavier than micky a huge difference in professional boxing constituting two or three weight classes despite mickys reservations his mother and brother agree so that they can all get the purse and micky is defeated micky retreats from the world and forms a relationship with charlene fleming a former college athlete who dropped out and became a bartender after several weeks alice arranges another fight for micky concerned it will turn out the same his mother and seven sisters blame charlene for his lack of motivation micky mentions he received an offer to be paid to train in las vegas but dicky says he will match the offer so he can keep training and working with his family dicky then tries to get money by posing his girlfriend as a prostitute and then once she picks up a client impersonating a police officer to steal the clients money this is foiled by the actual police and dicky is arrested after a chase and a fight with them micky tries to stop the police from beating his brother and a police officer brutally breaks his hand before arresting him at their arraignment micky is released but dicky is sent to jail micky washes his hands of dicky on the night of the hbo documentarys airing dickys family and dicky himself in prison are horrified to see it is called crack in america how crack addiction ruined dickys career and life dicky begins training and trying to get his life together in prison micky is lured back into boxing by his father who believes alice and his stepson dicky are bad influences the other members of his training team and a new manager sal lanano persuade micky to return to boxing with the explicit understanding that his mother and brother will no longer be involved they place micky in minor fights to help him regain his confidence he is then offered another major fight against an undefeated upandcoming boxer during a prison visit dicky advises micky on how best to work his opponent but micky feels his brother is being selfish and trying to restart his own failed career during the actual match micky is nearly overwhelmed but then implements his brothers advice and triumphs he earns the title shot for which his opponent was being groomed upon his release from prison dicky and his mother go to see micky train assuming things are as they were dicky prepares to spar with his brother but micky informs him that hes no longer allowed per mickys agreement with his current team in the ensuing argument in which micky chastises both factions of his family charlene and his trainer leave in disgust micky and dicky spar until micky knocks dicky down dicky storms off presumably to get high again and alice chides micky only to be sobered when he tells her that she has always favored dicky dicky returns to his crack house where he says goodbye to his friends and heads to charlenes apartment he tells her that micky needs both of them and they need to work together after bringing everyone back together the group goes to london for the title fight micky scores another upset victory and the welterweight title the film jumps a few years ahead with dicky crediting his brother as the creator of his own success|||||the welterweight title",
"what is john buckinghams job/profession|||||john buckingham ben chaplin a lonely st albans bank clerk orders a mailorder bride nadia nicole kidman from russia on the internet john is uncomfortable and shy but nadia is sexually bold though nadia cannot speak english and john cannot speak russian they soon bond later on a man she introduces as her cousin yuri mathieu kassovitz and his friend alexei vincent cassel turn up to celebrate her birthday alexei soon shows that he has a temper after a violent altercation alexei holds nadia hostage and demands a ransom from john john has grown to care for nadia and is forced to steal from the bank where he has worked for ten years after the ransom is paid he realises that he has been the victim of an elaborate con nadia yuri and alexei are criminals and alexei is actually nadias boyfriend john learns that the trio have carried out the same scam on men from switzerland and germany among others they take him prisoner strip him down to his underpants and tie him to a toilet in a motel he eventually manages to free himself and quickly learns that nadia has been left behind after alexei discovered she was pregnant john gets dressed and subsequently gets into a scrap with nadia who later reveals that she can indeed speak english and that her name is not nadia john takes nadia to turn her into the police hoping to clear his name as a wanted bank robber ultimately however he sympathises with her and decides against it he leaves her at the airport where she is kidnapped by alexei â who now wants nadia to have the baby john rescues her tying alexei to a chair they make common cause against the two russian men nadia informs john that her real name is sophia john disguised as alexei leaves for russia with sophia|||||bank clerk",
"who demonstrates/showed supernatural powers|||||in a louisiana assistedliving home in 1999 paul edgecomb begins to cry while watching the film top hat his companion elaine becomes concerned and paul explains to her that the film reminded him of the events of 1935 which took place when he was a prison officer in charge of death row and the states electric chair in 1935 paul supervises fellow officers brutus brutal howell dean stanton harry terwilliger and percy wetmore at cold mountain penitentiary on what they refer to as the green mile death row paul suffering from a severe bladder infection receives a physically imposing but mentally challenged black man john coffey into his custody john had been convicted of raping and murdering two little white girls one of the other inmates is a nativeamerican named arlen bitterbuck who is charged with murder and is the first to be executed percy demonstrates a severe sadistic and bullying streak but is beyond reproach he is the nephew of louisianas first lady but only paul is bold enough to defy this he is particularly abusive with inmate eduard del delacroix he breaks dels fingers with his baton steps on a pet mouse named mr jingles which del had adopted repeatedly calls him by a gay slur and ultimately sabotages his execution by failing to soak the sponge used to conduct electricity to dels head dels body explodes in flames and he dies screaming in pain john begins to demonstrate supernatural powers he cures pauls bladder infection resurrects mr jingles and heals melinda moores wife of the prisons chief warden of a brain tumor this last affliction he releases into percy who under its influence shoots another prisoner mass murderer william wharton dead wharton had from the moment of his arrival been a troublemaker he attacked the guards as he was being escorted into the block made mischief on two occasions later which caused paul to order him restrained in the blocks padded cell groped percy briefly racially insulted john and revealed psychically to john that he is in fact responsible for the crime for which john was unjustly condemned john then reveals the story psychically to paul but in so doing releases his supernatural energy into paul percy meanwhile is committed to the insane asylum which ironically wharton had come from and where he percy was hopeful of getting a job john tells paul distraught over the notion of executing an innocent man that he does in fact wish to die as he views the world as a cruel place mentioning that he had never before seen a movie john watches top hat with the other guards as a last request that night john is executed although he refuses the customary hood as he is afraid of the dark paul concludes his story by telling elaine that johns was the last execution that either he or brutus supervised following coffeys execution they both took jobs in the juvenile system elaine realizes that since he had a grown son in 1935 paul must be much older than he looks paul reveals that he is in fact 108 years of age and not only is he still alive so is mr jingles dels mouse paul then muses that if johns power could make a mouse live for as long as mr jingles has how much longer does he himself have left|||||john",
"what item does clara possess/have that makes jack suspicious/dobutful of her|||||jack george clooney a gunsmith and contract killer and his lover ingrid irina björklund are relaxing in sweden jack becomes alarmed by a trail of footprints in the snow and pulls ingrid towards shelter sniper gunshots ring out ingrid sees jack pull a gun from his pocket and shoot the approaching sniper knowing his identity is in jeopardy and with little hesitation jack shoots and kills ingrid he flees to rome and contacts his handler pavel johan leysen who insists that jack cannot stay in rome pavel sends him to castelvecchio a small town in the mountains of abruzzo jack becomes nervous and disposing of the cell phone pavel gave him goes to nearby castel del monte abruzzo instead while in abruzzo jack contacts pavel who sets him up with another job he meets mathilde thekla reuten who wants him to build a custom sniper rifle for an assassination he also begins patronizing a prostitute clara violante placido and they begin a relationship separate from her business duties jack meets with mathilde to test the weapon she is impressed by the craftsmanship but asks him to make a few more adjustments later jack realizes that he is being followed by an assassin from sweden whom he kills jack is tormented by dreams of the events in sweden and regrets killing ingrid his friendship and conversations with a local priest father benedetto paolo bonacelli encourage him to question his life choices both mathilde and clara notice jacks association with butterflies mathilde by his expertise in endangered butterflies and clara by his prominent tattoo when father benedetto tells jack he senses he lives in a special kind of hell a place without love jack starts to let himself feel love for clara and envisions a life with her jack talks to pavel one last time—asking how the swedes have found him in his growing fear he even suspects clara when he discovers a small pistol in her purse he questions clara about the gun and is satisfied with her explanation to defend herself since there have been newspaper reports of murdered prostitutes jack agrees to deliver the weapon to mathilde as his last job—but at the last moment he opens the briefcase and sabotages the rifle at the dropoff jack becomes suspicious that mathilde plans to kill him before anything can happen a busload of school children arrives mathilde gives jack his payment—a thick envelope full of cash—and the two separate as mathilde drives away pavel contacts her and asks if she has killed jack she tells him she has not but says she is following him and will kill him clara meets jack at a religious procession in town jack asks her to go away with him and she agrees mathilde tries to shoot jack from a nearby rooftop but the sabotaged rifle explodes in her face seeing mathilde fall from the roof jack gives clara the envelope full of cash and tells her to go to a river where they had picnicked and wait for him he runs to mathilde who is dying on the pavement and discovers that she also works for pavel as jack goes to meet clara he hears pavel behind him they exchange gunfire and pavel drops dead as jack drives to meet clara at the river he feels his abdomen and realizes he has been shot jack arrives at the picnic spot and as he sees clara he collapses clara screams and runs to the car a white butterfly flits skyward from jacks car|||||a pistol",
"what is grassous biggest/major goal/aim|||||pierre grassou de fougères is a mediocre painter who lives off painting forgeries commissioned by an old swindler and artdealer named elias magus grassou paints forgeries of works by titian raphael and other famous artists magus resells these forgeries for a large profit to members of the petite bourgeoisie who are incapable of appreciating good art monsieur vervelle a prosperous bottledealer obsessed with art is introduced to grassou by magus who introduces the painter as a grand master vervelle and his wife are enchanted with grassou and believe he would make a good match for their daughter virginie grassou is invited to villedavray where the vervelle mansion is garishly decorated and also includes a large collection of grassou’s work including the forgery of a titian grassou recognizes his own mediocrity but when it is discovered by the vervelles that the forgeries in their home were painted by grassou “far from denting his reputation multiples grassou’s value as an artist” and as a soninlaw since vervelle believes grassou to possess all the combined talent of rubens rembrandt terburg and titian despite his advantageous marriage grassou remains bitter that he is not a true artist “this painter a good father and a good husband is unable to eradicate from his heart a fatal thought namely that artists laugh at his work that his name is a term of contempt in the studios and that the feuilletons take no notice of his pictures but he still works on he aims for the academy where undoubtedly he will enter|||||to enter the academy",
"what creature/animal does jack have an affinity/empathy for|||||jack george clooney a gunsmith and contract killer and his lover ingrid irina björklund are relaxing in sweden jack becomes alarmed by a trail of footprints in the snow and pulls ingrid towards shelter sniper gunshots ring out ingrid sees jack pull a gun from his pocket and shoot the approaching sniper knowing his identity is in jeopardy and with little hesitation jack shoots and kills ingrid he flees to rome and contacts his handler pavel johan leysen who insists that jack cannot stay in rome pavel sends him to castelvecchio a small town in the mountains of abruzzo jack becomes nervous and disposing of the cell phone pavel gave him goes to nearby castel del monte abruzzo instead while in abruzzo jack contacts pavel who sets him up with another job he meets mathilde thekla reuten who wants him to build a custom sniper rifle for an assassination he also begins patronizing a prostitute clara violante placido and they begin a relationship separate from her business duties jack meets with mathilde to test the weapon she is impressed by the craftsmanship but asks him to make a few more adjustments later jack realizes that he is being followed by an assassin from sweden whom he kills jack is tormented by dreams of the events in sweden and regrets killing ingrid his friendship and conversations with a local priest father benedetto paolo bonacelli encourage him to question his life choices both mathilde and clara notice jacks association with butterflies mathilde by his expertise in endangered butterflies and clara by his prominent tattoo when father benedetto tells jack he senses he lives in a special kind of hell a place without love jack starts to let himself feel love for clara and envisions a life with her jack talks to pavel one last time—asking how the swedes have found him in his growing fear he even suspects clara when he discovers a small pistol in her purse he questions clara about the gun and is satisfied with her explanation to defend herself since there have been newspaper reports of murdered prostitutes jack agrees to deliver the weapon to mathilde as his last job—but at the last moment he opens the briefcase and sabotages the rifle at the dropoff jack becomes suspicious that mathilde plans to kill him before anything can happen a busload of school children arrives mathilde gives jack his payment—a thick envelope full of cash—and the two separate as mathilde drives away pavel contacts her and asks if she has killed jack she tells him she has not but says she is following him and will kill him clara meets jack at a religious procession in town jack asks her to go away with him and she agrees mathilde tries to shoot jack from a nearby rooftop but the sabotaged rifle explodes in her face seeing mathilde fall from the roof jack gives clara the envelope full of cash and tells her to go to a river where they had picnicked and wait for him he runs to mathilde who is dying on the pavement and discovers that she also works for pavel as jack goes to meet clara he hears pavel behind him they exchange gunfire and pavel drops dead as jack drives to meet clara at the river he feels his abdomen and realizes he has been shot jack arrives at the picnic spot and as he sees clara he collapses clara screams and runs to the car a white butterfly flits skyward from jacks car|||||butterflies",
"who warns/advices settlers of the pictish attacks/ambushes|||||from conan the warrior isbn 0441114652 the foreword to the story tells of his travels to punt with muriela refers to a scam perpetrated against worshippers of an ivory goddess and then on to zembabwei where he joins a trading caravan on its way to shem around 40 now conan visits his homeland and finds his old friends are fathers bored conan sets off for the bossonian marches and becomes a scout at fort tuscelan on the black river naturally there is a war going on a young settler named balthus encounters conan in the forests slaying a forest devil accompanying the young man back to the fort conan finds the body of a merchant ensorcelled by a pictish wizard named zogar sag and slain by a swamp demon the fort tuscelan commander valannus is a desperate man and asks conan to slay zogar sag before he raises the picts against the whole borderlands taking a hand picked team of scouts and balthus conan sets off stealthily in canoes balthus is captured and most of conans men slaughtered in an ambush balthus and one of the scouts are tied to stakes and the scout is sacrificed by zogar sag to one of his jungle creatures before balthus can meet a similar fate conan sets the pictish village on fire and the two flee into the woods conan tells balthus of the cult of jhebbal sag now forgotten by most once all living things worshipped him when men and beasts spoke the same language over time men and most beasts forgot his worship zogar sag has not however and can control those few animals and creatures who also remember and they are on conans trail now conan is able to neutralize them using a symbol he once noticed and the pair hurry to return to the fort to warn them of the impending pictish assault but they are too late the picts already are all around the fort and furious fighting is going on the number of picts ensures that eventually the fort will be overwhelmed and the defenders slaughtered the only thing left to do is warn the settlers to flee while the picts are busy with the fort otherwise they will be slaughtered too conan and balthus go to warn the settlers that the picts have crossed the river and are raiding they are joined by slasher a feral dog formerly owned by a settler who had been slain by the picts balthus is sent on to warn settlers of the coming pict raid and conan parts from him to warn a group of settler who had gone to gather salt balthus warns women and children to leave their huts and flee when a band of picts arrives who move quicker and might overtake the women balthus stays behind to cover their escape accompanied by slasher he makes a stand against the coming pict raiders first shooting arrows from concealment and then in a furious face to face battle the man and dogs sacrifice delays the picts and gives the settlers time to reach safety conan manages to warn the saltgathering party in time but finds he has been marked for death by the gods of darkness for misusing the symbol of jhebbal sag in the end conan triumphs but the fort is lost and so is the entire province the story ends in a tavern a survivor tells conan about the courageous act of balthus and slasher and how their final stand had delayed the picts just barely long enough for the settlers to reach safety upon hearing of the fight conan vowed to take the heads of ten picts to pay for balthus sacrifice along with seven heads for the dog who was a better warrior than many a man|||||balthus",
"what is esthers real/actual name|||||kate and john coleman vera farmiga and peter sarsgaard are experiencing strains in their marriage after their third child is stillborn the loss is particularly hard on kate who is also recovering from alcoholism the couple decides to adopt a 9yearold russian girl named esther isabelle fuhrman from a local orphanage while kate and johns deaf daughter max aryana engineer embraces esther immediately their son daniel jimmy bennett is less welcoming kate suspects that there might be problems in esthers background when esthers knowledge of sex extends beyond her age her suspicions deepen when esther injures another girl who had bullied her at school kate is further alarmed when sister abigail c c h pounder the head of the orphanage warns her that bad things happen when esther is around esther overhears this and plots to use max to distract sister abigail and then kill her esther pushes max into the path of sister abigails car forcing her to swerve off the road esther then kills sister abigail with a hammer and forces max to help her hide the weapon in their tree house kate is convinced that something is very wrong with esther but john does not believe her kate finds esthers hidden bible and discovers that it came from the saarne institute in estonia a mental hospital she emails a picture of esther to them and asks for more information when daniel learns about sister abigails death he tells max of his plan to retrieve the hammer to prove esthers guilt esther overhears this and confronts daniel at the tree house then sets it ablaze to kill him and destroy the evidence daniel falls from the tree trying to escape and is knocked unconscious esther attempts to finish him off but max stops her while daniel is hospitalized esther slips into his room and smothers him with a pillow stopping his heart doctors quickly revive him kate realizes what happened and attacks esther but is restrained and sedated that night a provocativelydressed esther tries to seduce a drunken john who finally realizes kate was right he tells esther that her future with the colemans is at stake upsetting her kate receives a call from dr v채rava karel roden the director of the saarne institute who reveals that esther is actually a 33yearold woman named leena klammer who has hypopituitarism a hormone disorder that stunted her physical growth and that she has spent most of her life posing as a little girl and had people adopt her with the goal of seducing the father and having a sexual relationship with him the doctor reports that leena has murdered at least seven people the previous family she was adopted by was killed by her because the father had rejected her sexual advances kate tries to get back home to stop esther and prevent her family from suffering the same fate leena flies into a rage after being spurned by john and ransacks her room removing her little girl makeup revealing the scars from a straitjacket and weathered skin leena stabs and kills john max witnesses this and hides kate rushes home and finds john dead leena gets a gun from johns safe and shoots kate in the arm kate manages to take the gun and flee with max leena finds kate and max near a frozen pond she lunges at kate hurling them both into the ice kate climbs partially out of the pond leena reverting to her esther persona begs kate not to let her die while hiding a knife behind her back kate kills leena by kicking her in the face breaking her neck max and kate are met by the police moments after|||||lenna klammer",
"what is the purpose/objective of the fiddlefest concert|||||the film opens with violinist roberta guaspari having been deserted by her us navy husband and feeling devastated almost suicidal encouraged by her mother she attempts to rebuild her life and a friend from student days recommends her to the head teacher of a school in the tough new york city area of east harlem despite a degree in music education she has little experience in actual music teaching but shes taken on as a substitute violin teacher with a combination of toughness and determination she inspires a group of kids and their initially skeptical parents the program slowly develops and attracts publicity ten years later the string program is still running successfully at three schools but suddenly the school budget is cut and roberta is out of a job determined to fight the cuts she enlists the support of former pupils parents and teachers and plans a grand fundraising concert fiddlefest to raise money so that the program can continue but with a few weeks to go and all participants furiously rehearsing they lose the venue fortunately the husband of a publicist friend is a violinist in the guarneri quartet and he enlists the support of other wellknown musicians including isaac stern and itzhak perlman they arrange for the concert to be mounted at carnegie hall other famous musicians including mark oconnor michael tree charles veal jr arnold steinhardt karen briggs sandra park diane monroe and joshua bell join in the performance which is a resounding success the films end credits declare that the opus 118 program is still running successfully they also report that the schools funding was restored during the making of the film|||||raise money to save the program",
"which disability/condition is max afflicted/troubled with|||||kate and john coleman vera farmiga and peter sarsgaard are experiencing strains in their marriage after their third child is stillborn the loss is particularly hard on kate who is also recovering from alcoholism the couple decides to adopt a 9yearold russian girl named esther isabelle fuhrman from a local orphanage while kate and johns deaf daughter max aryana engineer embraces esther immediately their son daniel jimmy bennett is less welcoming kate suspects that there might be problems in esthers background when esthers knowledge of sex extends beyond her age her suspicions deepen when esther injures another girl who had bullied her at school kate is further alarmed when sister abigail c c h pounder the head of the orphanage warns her that bad things happen when esther is around esther overhears this and plots to use max to distract sister abigail and then kill her esther pushes max into the path of sister abigails car forcing her to swerve off the road esther then kills sister abigail with a hammer and forces max to help her hide the weapon in their tree house kate is convinced that something is very wrong with esther but john does not believe her kate finds esthers hidden bible and discovers that it came from the saarne institute in estonia a mental hospital she emails a picture of esther to them and asks for more information when daniel learns about sister abigails death he tells max of his plan to retrieve the hammer to prove esthers guilt esther overhears this and confronts daniel at the tree house then sets it ablaze to kill him and destroy the evidence daniel falls from the tree trying to escape and is knocked unconscious esther attempts to finish him off but max stops her while daniel is hospitalized esther slips into his room and smothers him with a pillow stopping his heart doctors quickly revive him kate realizes what happened and attacks esther but is restrained and sedated that night a provocativelydressed esther tries to seduce a drunken john who finally realizes kate was right he tells esther that her future with the colemans is at stake upsetting her kate receives a call from dr v채rava karel roden the director of the saarne institute who reveals that esther is actually a 33yearold woman named leena klammer who has hypopituitarism a hormone disorder that stunted her physical growth and that she has spent most of her life posing as a little girl and had people adopt her with the goal of seducing the father and having a sexual relationship with him the doctor reports that leena has murdered at least seven people the previous family she was adopted by was killed by her because the father had rejected her sexual advances kate tries to get back home to stop esther and prevent her family from suffering the same fate leena flies into a rage after being spurned by john and ransacks her room removing her little girl makeup revealing the scars from a straitjacket and weathered skin leena stabs and kills john max witnesses this and hides kate rushes home and finds john dead leena gets a gun from johns safe and shoots kate in the arm kate manages to take the gun and flee with max leena finds kate and max near a frozen pond she lunges at kate hurling them both into the ice kate climbs partially out of the pond leena reverting to her esther persona begs kate not to let her die while hiding a knife behind her back kate kills leena by kicking her in the face breaking her neck max and kate are met by the police moments after|||||she is deaf"]

final_questions, final_changes, ans = process_questions(synonyms)

In [63]:
len(contexts_na)

159

In [64]:
name='Replace some words with synonyms'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle synonyms in questions"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(final_questions, final_changes, name, capability, test_type, n_description, t_description, c_description)

the welterweight title
bank clerk
john
a pistol
to enter the academy
butterflies
balthus
lenna klammer
raise money to save the program
she is deaf


0

In [65]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'what is the saarne institute',
    'question': "what's the saarne institute",
    'context': 'kate and john coleman vera farmiga and peter sarsga

In [66]:
#antonym questions nq
antonyms = ["who is jacks lover/enemy|||||jack george clooney a gunsmith and contract killer and his lover ingrid irina björklund are relaxing in sweden jack becomes alarmed by a trail of footprints in the snow and pulls ingrid towards shelter sniper gunshots ring out ingrid sees jack pull a gun from his pocket and shoot the approaching sniper knowing his identity is in jeopardy and with little hesitation jack shoots and kills ingrid he flees to rome and contacts his handler pavel johan leysen who insists that jack cannot stay in rome pavel sends him to castelvecchio a small town in the mountains of abruzzo jack becomes nervous and disposing of the cell phone pavel gave him goes to nearby castel del monte abruzzo instead while in abruzzo jack contacts pavel who sets him up with another job he meets mathilde thekla reuten who wants him to build a custom sniper rifle for an assassination he also begins patronizing a prostitute clara violante placido and they begin a relationship separate from her business duties jack meets with mathilde to test the weapon she is impressed by the craftsmanship but asks him to make a few more adjustments later jack realizes that he is being followed by an assassin from sweden whom he kills jack is tormented by dreams of the events in sweden and regrets killing ingrid his friendship and conversations with a local priest father benedetto paolo bonacelli encourage him to question his life choices both mathilde and clara notice jacks association with butterflies mathilde by his expertise in endangered butterflies and clara by his prominent tattoo when father benedetto tells jack he senses he lives in a special kind of hell a place without love jack starts to let himself feel love for clara and envisions a life with her jack talks to pavel one last time—asking how the swedes have found him in his growing fear he even suspects clara when he discovers a small pistol in her purse he questions clara about the gun and is satisfied with her explanation to defend herself since there have been newspaper reports of murdered prostitutes jack agrees to deliver the weapon to mathilde as his last job—but at the last moment he opens the briefcase and sabotages the rifle at the dropoff jack becomes suspicious that mathilde plans to kill him before anything can happen a busload of school children arrives mathilde gives jack his payment—a thick envelope full of cash—and the two separate as mathilde drives away pavel contacts her and asks if she has killed jack she tells him she has not but says she is following him and will kill him clara meets jack at a religious procession in town jack asks her to go away with him and she agrees mathilde tries to shoot jack from a nearby rooftop but the sabotaged rifle explodes in her face seeing mathilde fall from the roof jack gives clara the envelope full of cash and tells her to go to a river where they had picnicked and wait for him he runs to mathilde who is dying on the pavement and discovers that she also works for pavel as jack goes to meet clara he hears pavel behind him they exchange gunfire and pavel drops dead as jack drives to meet clara at the river he feels his abdomen and realizes he has been shot jack arrives at the picnic spot and as he sees clara he collapses clara screams and runs to the car a white butterfly flits skyward from jacks car|||||ingrid",
"what is grassous biggest/smallest goal|||||pierre grassou de fougères is a mediocre painter who lives off painting forgeries commissioned by an old swindler and artdealer named elias magus grassou paints forgeries of works by titian raphael and other famous artists magus resells these forgeries for a large profit to members of the petite bourgeoisie who are incapable of appreciating good art monsieur vervelle a prosperous bottledealer obsessed with art is introduced to grassou by magus who introduces the painter as a grand master vervelle and his wife are enchanted with grassou and believe he would make a good match for their daughter virginie grassou is invited to villedavray where the vervelle mansion is garishly decorated and also includes a large collection of grassou’s work including the forgery of a titian grassou recognizes his own mediocrity but when it is discovered by the vervelles that the forgeries in their home were painted by grassou “far from denting his reputation multiples grassou’s value as an artist” and as a soninlaw since vervelle believes grassou to possess all the combined talent of rubens rembrandt terburg and titian despite his advantageous marriage grassou remains bitter that he is not a true artist “this painter a good father and a good husband is unable to eradicate from his heart a fatal thought namely that artists laugh at his work that his name is a term of contempt in the studios and that the feuilletons take no notice of his pictures but he still works on he aims for the academy where undoubtedly he will enter|||||to enter the academy",
"how many pict heads does conan say he will take/give for balthuss sacrafice|||||from conan the warrior isbn 0441114652 the foreword to the story tells of his travels to punt with muriela refers to a scam perpetrated against worshippers of an ivory goddess and then on to zembabwei where he joins a trading caravan on its way to shem around 40 now conan visits his homeland and finds his old friends are fathers bored conan sets off for the bossonian marches and becomes a scout at fort tuscelan on the black river naturally there is a war going on a young settler named balthus encounters conan in the forests slaying a forest devil accompanying the young man back to the fort conan finds the body of a merchant ensorcelled by a pictish wizard named zogar sag and slain by a swamp demon the fort tuscelan commander valannus is a desperate man and asks conan to slay zogar sag before he raises the picts against the whole borderlands taking a hand picked team of scouts and balthus conan sets off stealthily in canoes balthus is captured and most of conans men slaughtered in an ambush balthus and one of the scouts are tied to stakes and the scout is sacrificed by zogar sag to one of his jungle creatures before balthus can meet a similar fate conan sets the pictish village on fire and the two flee into the woods conan tells balthus of the cult of jhebbal sag now forgotten by most once all living things worshipped him when men and beasts spoke the same language over time men and most beasts forgot his worship zogar sag has not however and can control those few animals and creatures who also remember and they are on conans trail now conan is able to neutralize them using a symbol he once noticed and the pair hurry to return to the fort to warn them of the impending pictish assault but they are too late the picts already are all around the fort and furious fighting is going on the number of picts ensures that eventually the fort will be overwhelmed and the defenders slaughtered the only thing left to do is warn the settlers to flee while the picts are busy with the fort otherwise they will be slaughtered too conan and balthus go to warn the settlers that the picts have crossed the river and are raiding they are joined by slasher a feral dog formerly owned by a settler who had been slain by the picts balthus is sent on to warn settlers of the coming pict raid and conan parts from him to warn a group of settler who had gone to gather salt balthus warns women and children to leave their huts and flee when a band of picts arrives who move quicker and might overtake the women balthus stays behind to cover their escape accompanied by slasher he makes a stand against the coming pict raiders first shooting arrows from concealment and then in a furious face to face battle the man and dogs sacrifice delays the picts and gives the settlers time to reach safety conan manages to warn the saltgathering party in time but finds he has been marked for death by the gods of darkness for misusing the symbol of jhebbal sag in the end conan triumphs but the fort is lost and so is the entire province the story ends in a tavern a survivor tells conan about the courageous act of balthus and slasher and how their final stand had delayed the picts just barely long enough for the settlers to reach safety upon hearing of the fight conan vowed to take the heads of ten picts to pay for balthus sacrifice along with seven heads for the dog who was a better warrior than many a man|||||ten",
"what does mary do when she hears that henrys consumption/saving has worsened/improved|||||mary begins with a description of the conventional and loveless marriage between the heroines mother and father eliza marys mother is obsessed with novels rarely considers anyone but herself and favours marys brother she neglects her daughter who educates herself using only books and the natural world ignored by her family mary devotes much of her time to charity when her brother suddenly dies leaving mary heir to the familys fortune her mother finally takes an interest in her she is taught accomplishments such as dancing that will attract suitors however marys mother soon sickens and requests on her deathbed that mary wed charles a wealthy man she has never met stunned and unable to refuse mary agrees immediately after the ceremony charles departs for the continent to escape a family who does not share her values mary befriends ann a local girl who educates her still further mary becomes quite attached to ann who is in the grip of an unrequited love and does not reciprocate marys feelings anns family falls into poverty and is on the brink of losing their home but mary is able to pay off their debts after her marriage to charles gives her limited control over her money ann becomes consumptive and mary travels with her to lisbon in hopes of nursing her back to health there they are introduced to henry who is also trying to regain his health ann dies and mary is griefstricken henry and mary subsequently fall in love but are forced to return to england separately mary depressed by her marriage to charles and bereft of both ann and henry remains unsettled until she hears that henrys consumption has worsened she rushes to his side and cares for him until he dies at the end of the novel charles returns from europe he and mary establish something of a life together but mary is unhealthy and can barely stand to be in the same room with her husband the last few lines of the novel imply that she will die young|||||she returns to his side and cares for him until he passes away",
"why is grassou still resentful/satisfied despite his potentially advantageous/disadvantageous marriage|||||pierre grassou de fougères is a mediocre painter who lives off painting forgeries commissioned by an old swindler and artdealer named elias magus grassou paints forgeries of works by titian raphael and other famous artists magus resells these forgeries for a large profit to members of the petite bourgeoisie who are incapable of appreciating good art monsieur vervelle a prosperous bottledealer obsessed with art is introduced to grassou by magus who introduces the painter as a grand master vervelle and his wife are enchanted with grassou and believe he would make a good match for their daughter virginie grassou is invited to villedavray where the vervelle mansion is garishly decorated and also includes a large collection of grassou’s work including the forgery of a titian grassou recognizes his own mediocrity but when it is discovered by the vervelles that the forgeries in their home were painted by grassou “far from denting his reputation multiples grassou’s value as an artist” and as a soninlaw since vervelle believes grassou to possess all the combined talent of rubens rembrandt terburg and titian despite his advantageous marriage grassou remains bitter that he is not a true artist “this painter a good father and a good husband is unable to eradicate from his heart a fatal thought namely that artists laugh at his work that his name is a term of contempt in the studios and that the feuilletons take no notice of his pictures but he still works on he aims for the academy where undoubtedly he will enter|||||because he is not a true artist",
"where does the end/start of the story take place|||||from conan the warrior isbn 0441114652 the foreword to the story tells of his travels to punt with muriela refers to a scam perpetrated against worshippers of an ivory goddess and then on to zembabwei where he joins a trading caravan on its way to shem around 40 now conan visits his homeland and finds his old friends are fathers bored conan sets off for the bossonian marches and becomes a scout at fort tuscelan on the black river naturally there is a war going on a young settler named balthus encounters conan in the forests slaying a forest devil accompanying the young man back to the fort conan finds the body of a merchant ensorcelled by a pictish wizard named zogar sag and slain by a swamp demon the fort tuscelan commander valannus is a desperate man and asks conan to slay zogar sag before he raises the picts against the whole borderlands taking a hand picked team of scouts and balthus conan sets off stealthily in canoes balthus is captured and most of conans men slaughtered in an ambush balthus and one of the scouts are tied to stakes and the scout is sacrificed by zogar sag to one of his jungle creatures before balthus can meet a similar fate conan sets the pictish village on fire and the two flee into the woods conan tells balthus of the cult of jhebbal sag now forgotten by most once all living things worshipped him when men and beasts spoke the same language over time men and most beasts forgot his worship zogar sag has not however and can control those few animals and creatures who also remember and they are on conans trail now conan is able to neutralize them using a symbol he once noticed and the pair hurry to return to the fort to warn them of the impending pictish assault but they are too late the picts already are all around the fort and furious fighting is going on the number of picts ensures that eventually the fort will be overwhelmed and the defenders slaughtered the only thing left to do is warn the settlers to flee while the picts are busy with the fort otherwise they will be slaughtered too conan and balthus go to warn the settlers that the picts have crossed the river and are raiding they are joined by slasher a feral dog formerly owned by a settler who had been slain by the picts balthus is sent on to warn settlers of the coming pict raid and conan parts from him to warn a group of settler who had gone to gather salt balthus warns women and children to leave their huts and flee when a band of picts arrives who move quicker and might overtake the women balthus stays behind to cover their escape accompanied by slasher he makes a stand against the coming pict raiders first shooting arrows from concealment and then in a furious face to face battle the man and dogs sacrifice delays the picts and gives the settlers time to reach safety conan manages to warn the saltgathering party in time but finds he has been marked for death by the gods of darkness for misusing the symbol of jhebbal sag in the end conan triumphs but the fort is lost and so is the entire province the story ends in a tavern a survivor tells conan about the courageous act of balthus and slasher and how their final stand had delayed the picts just barely long enough for the settlers to reach safety upon hearing of the fight conan vowed to take the heads of ten picts to pay for balthus sacrifice along with seven heads for the dog who was a better warrior than many a man|||||in a tavern",
"what does mary devote/waste much of her time to|||||mary begins with a description of the conventional and loveless marriage between the heroines mother and father eliza marys mother is obsessed with novels rarely considers anyone but herself and favours marys brother she neglects her daughter who educates herself using only books and the natural world ignored by her family mary devotes much of her time to charity when her brother suddenly dies leaving mary heir to the familys fortune her mother finally takes an interest in her she is taught accomplishments such as dancing that will attract suitors however marys mother soon sickens and requests on her deathbed that mary wed charles a wealthy man she has never met stunned and unable to refuse mary agrees immediately after the ceremony charles departs for the continent to escape a family who does not share her values mary befriends ann a local girl who educates her still further mary becomes quite attached to ann who is in the grip of an unrequited love and does not reciprocate marys feelings anns family falls into poverty and is on the brink of losing their home but mary is able to pay off their debts after her marriage to charles gives her limited control over her money ann becomes consumptive and mary travels with her to lisbon in hopes of nursing her back to health there they are introduced to henry who is also trying to regain his health ann dies and mary is griefstricken henry and mary subsequently fall in love but are forced to return to england separately mary depressed by her marriage to charles and bereft of both ann and henry remains unsettled until she hears that henrys consumption has worsened she rushes to his side and cares for him until he dies at the end of the novel charles returns from europe he and mary establish something of a life together but mary is unhealthy and can barely stand to be in the same room with her husband the last few lines of the novel imply that she will die young|||||charity",
"why do the gods of darkness/brightness mark conan for death|||||from conan the warrior isbn 0441114652 the foreword to the story tells of his travels to punt with muriela refers to a scam perpetrated against worshippers of an ivory goddess and then on to zembabwei where he joins a trading caravan on its way to shem around 40 now conan visits his homeland and finds his old friends are fathers bored conan sets off for the bossonian marches and becomes a scout at fort tuscelan on the black river naturally there is a war going on a young settler named balthus encounters conan in the forests slaying a forest devil accompanying the young man back to the fort conan finds the body of a merchant ensorcelled by a pictish wizard named zogar sag and slain by a swamp demon the fort tuscelan commander valannus is a desperate man and asks conan to slay zogar sag before he raises the picts against the whole borderlands taking a hand picked team of scouts and balthus conan sets off stealthily in canoes balthus is captured and most of conans men slaughtered in an ambush balthus and one of the scouts are tied to stakes and the scout is sacrificed by zogar sag to one of his jungle creatures before balthus can meet a similar fate conan sets the pictish village on fire and the two flee into the woods conan tells balthus of the cult of jhebbal sag now forgotten by most once all living things worshipped him when men and beasts spoke the same language over time men and most beasts forgot his worship zogar sag has not however and can control those few animals and creatures who also remember and they are on conans trail now conan is able to neutralize them using a symbol he once noticed and the pair hurry to return to the fort to warn them of the impending pictish assault but they are too late the picts already are all around the fort and furious fighting is going on the number of picts ensures that eventually the fort will be overwhelmed and the defenders slaughtered the only thing left to do is warn the settlers to flee while the picts are busy with the fort otherwise they will be slaughtered too conan and balthus go to warn the settlers that the picts have crossed the river and are raiding they are joined by slasher a feral dog formerly owned by a settler who had been slain by the picts balthus is sent on to warn settlers of the coming pict raid and conan parts from him to warn a group of settler who had gone to gather salt balthus warns women and children to leave their huts and flee when a band of picts arrives who move quicker and might overtake the women balthus stays behind to cover their escape accompanied by slasher he makes a stand against the coming pict raiders first shooting arrows from concealment and then in a furious face to face battle the man and dogs sacrifice delays the picts and gives the settlers time to reach safety conan manages to warn the saltgathering party in time but finds he has been marked for death by the gods of darkness for misusing the symbol of jhebbal sag in the end conan triumphs but the fort is lost and so is the entire province the story ends in a tavern a survivor tells conan about the courageous act of balthus and slasher and how their final stand had delayed the picts just barely long enough for the settlers to reach safety upon hearing of the fight conan vowed to take the heads of ten picts to pay for balthus sacrifice along with seven heads for the dog who was a better warrior than many a man|||||he misused the jhebbal sag symbol",
"what does mathilde ask jack to build/destroy|||||jack george clooney a gunsmith and contract killer and his lover ingrid irina björklund are relaxing in sweden jack becomes alarmed by a trail of footprints in the snow and pulls ingrid towards shelter sniper gunshots ring out ingrid sees jack pull a gun from his pocket and shoot the approaching sniper knowing his identity is in jeopardy and with little hesitation jack shoots and kills ingrid he flees to rome and contacts his handler pavel johan leysen who insists that jack cannot stay in rome pavel sends him to castelvecchio a small town in the mountains of abruzzo jack becomes nervous and disposing of the cell phone pavel gave him goes to nearby castel del monte abruzzo instead while in abruzzo jack contacts pavel who sets him up with another job he meets mathilde thekla reuten who wants him to build a custom sniper rifle for an assassination he also begins patronizing a prostitute clara violante placido and they begin a relationship separate from her business duties jack meets with mathilde to test the weapon she is impressed by the craftsmanship but asks him to make a few more adjustments later jack realizes that he is being followed by an assassin from sweden whom he kills jack is tormented by dreams of the events in sweden and regrets killing ingrid his friendship and conversations with a local priest father benedetto paolo bonacelli encourage him to question his life choices both mathilde and clara notice jacks association with butterflies mathilde by his expertise in endangered butterflies and clara by his prominent tattoo when father benedetto tells jack he senses he lives in a special kind of hell a place without love jack starts to let himself feel love for clara and envisions a life with her jack talks to pavel one last time—asking how the swedes have found him in his growing fear he even suspects clara when he discovers a small pistol in her purse he questions clara about the gun and is satisfied with her explanation to defend herself since there have been newspaper reports of murdered prostitutes jack agrees to deliver the weapon to mathilde as his last job—but at the last moment he opens the briefcase and sabotages the rifle at the dropoff jack becomes suspicious that mathilde plans to kill him before anything can happen a busload of school children arrives mathilde gives jack his payment—a thick envelope full of cash—and the two separate as mathilde drives away pavel contacts her and asks if she has killed jack she tells him she has not but says she is following him and will kill him clara meets jack at a religious procession in town jack asks her to go away with him and she agrees mathilde tries to shoot jack from a nearby rooftop but the sabotaged rifle explodes in her face seeing mathilde fall from the roof jack gives clara the envelope full of cash and tells her to go to a river where they had picnicked and wait for him he runs to mathilde who is dying on the pavement and discovers that she also works for pavel as jack goes to meet clara he hears pavel behind him they exchange gunfire and pavel drops dead as jack drives to meet clara at the river he feels his abdomen and realizes he has been shot jack arrives at the picnic spot and as he sees clara he collapses clara screams and runs to the car a white butterfly flits skyward from jacks car|||||a custom sniper rifle",
"when mary was younger how did her mother fill her free/occupied time|||||mary begins with a description of the conventional and loveless marriage between the heroines mother and father eliza marys mother is obsessed with novels rarely considers anyone but herself and favours marys brother she neglects her daughter who educates herself using only books and the natural world ignored by her family mary devotes much of her time to charity when her brother suddenly dies leaving mary heir to the familys fortune her mother finally takes an interest in her she is taught accomplishments such as dancing that will attract suitors however marys mother soon sickens and requests on her deathbed that mary wed charles a wealthy man she has never met stunned and unable to refuse mary agrees immediately after the ceremony charles departs for the continent to escape a family who does not share her values mary befriends ann a local girl who educates her still further mary becomes quite attached to ann who is in the grip of an unrequited love and does not reciprocate marys feelings anns family falls into poverty and is on the brink of losing their home but mary is able to pay off their debts after her marriage to charles gives her limited control over her money ann becomes consumptive and mary travels with her to lisbon in hopes of nursing her back to health there they are introduced to henry who is also trying to regain his health ann dies and mary is griefstricken henry and mary subsequently fall in love but are forced to return to england separately mary depressed by her marriage to charles and bereft of both ann and henry remains unsettled until she hears that henrys consumption has worsened she rushes to his side and cares for him until he dies at the end of the novel charles returns from europe he and mary establish something of a life together but mary is unhealthy and can barely stand to be in the same room with her husband the last few lines of the novel imply that she will die young|||||she was obsessed with novels"
]
final_questions, final_changes, ans = process_questions(antonyms)

In [67]:
name='Replace some words with antonym in the question'
test_type = "INV"
capability='Robustness'
n_description = "Test's model's ability to handle antonyms in questions"
c_description = capabilities[capability]
t_description = test_types[test_type]
add_test(final_questions, final_changes, name, capability, test_type, n_description, t_description, c_description)

ingrid
to enter the academy
ten
she returns to his side and cares for him until he passes away
because he is not a true artist
in a tavern
charity
he misused the jhebbal sag symbol
a custom sniper rifle
she was obsessed with novels


0

In [68]:
json_dicts[0]["tests"]

[{'test_type': 'INV',
  'capability': 'Robustness',
  'test_name': 'Question contractions',
  'test_name_description': "Test's model's ability to handle contraction in questions (whether contractions (what is -> what's) in the questions changes the model's output)",
  'test_type_description': 'Invariance in short INV test is inspired from metamorphic testing of software engineering. For this type of test label-preserving perturbations to inputs are applied and the expected prediction generated by the model should remain the same. INV tests can be performed with unlabeled data since they do not depend on true labels.',
  'capability_description': "This type of test capability test's model's capability to handle small changes in the input (typos, irrelevant sentence addition, question contraction etc.).",
  'test_cases': [{'original_question': 'what is the saarne institute',
    'question': "what's the saarne institute",
    'context': 'kate and john coleman vera farmiga and peter sarsga

In [69]:
import json
for i in range(len(selected_skills)):
    file_name = json_file_names[i]
    json_object = json.dumps(json_dicts[i], indent = 4)
    with open(file_name, "w") as outfile:
        outfile.write(json_object)